**Nama : Khairah Michiko D.W**

**NRP : 5054231028**

**Rekayasa Kecerdasan Artifisial**

**import library**

In [52]:
%pip install ipythonblocks
%pip install qpsolvers
from utils import *
from logic import *
from notebook import psource

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### membuat prop KB ###

In [7]:
P = ['P11', 'P12', 'P13', 'P14', 'P21', 'P22', 'P23', 'P24', 'P31', 'P32', 'P33', 'P34', 'P41', 'P42', 'P43', 'P44']
B = ['B11', 'B12', 'B13', 'B14', 'B21', 'B22', 'B23', 'B24', 'B31', 'B32', 'B33', 'B34', 'B41', 'B42', 'B43', 'B44']
S = ['S11', 'S12', 'S13', 'S14', 'S21', 'S22', 'S23', 'S24', 'S31', 'S32', 'S33', 'S34', 'S41', 'S42', 'S43', 'S44']
W = ['W11', 'W12', 'W13', 'W14', 'W21', 'W22', 'W23', 'W24', 'W31', 'W32', 'W33', 'W34', 'W41', 'W42', 'W43', 'W44']
Gold = 'Gold23'

### fungsi fungsi yang digunakan ##

In [9]:
psource(to_cnf)

In [63]:
psource(pl_resolution)

In [11]:
# Inisialisasi knowledge base untuk Wumpus
wumpus_kb = PropKB()

### proses pengubahan klausa menjadi Conjunctive Normal Form (CNF) ###

**mendefinisikan semua klausa berdasarkan masalah wumpus**

In [13]:
# Mendefinisikan semua klausa berdasarkan masalah Wumpus
clauses = [
    f"{B[4]} <=> ({P[0]} | {P[5]} | {P[10]})",  # B21 <=> Pit di (1.1, 2.2, 3.1)
    f"{B[12]} <=> ({P[10]} | {P[13]})",         # B41 <=> Pit di (3.1, 4.2)
    f"{B[9]} <=> ({P[10]} | {P[5]} | {P[9]})",  # B32 <=> Pit di (3.3, 2.2, 3.1)
    f"{B[5]} <=> ({P[5]} | {P[2]} | {P[7]})",   # B23 <=> Pit di (2.2, 1.3, 2.4)
    f"{B[14]} <=> ({P[13]} | {P[10]} | {P[15]})", # B43 <=> Pit di (4.2, 3.3, 4.4)
    f"{B[10]} <=> ({P[10]} | {P[7]} | {P[15]})", # B34 <=> Pit di (3.3, 2.4, 4.4)
    
    # Stench (Wumpus)
    f"{S[1]} <=> ({W[2]} | {W[0]} | {W[5]})",   # S12 <=> Wumpus di (1.3, 1.1, 2.2)
    f"{S[5]} <=> ({W[13]} | {W[5]} | {W[7]} | {W[2]})", # S23 <=> Wumpus di (3.3, 2.2, 4.2, 1.3)
    f"{S[3]} <=> ({W[2]} | {W[7]})",            # S14 <=> Wumpus di (1.3, 2.4)

    # Kondisi yang diketahui
    f"~{P[0]}",  # Tidak ada Pit di (1.1)
    f"~{W[0]}",  # Tidak ada Wumpus di (1.1)
    f"{Gold}",   # Ada emas di (2.3)
    f"({P[0]} & ~{W[0]})",  # Ada Pit di (1.1) tapi tidak ada Wumpus di sana
    f"({P[5]} & ~{P[7]})"   # Ada Pit di (2.2) tapi tidak ada Pit di (2.4)
]

### memeriksa klausa yang disimpan dalam KB ###

In [15]:
# Menampilkan klausa
for clause in clauses:
    print(clause)

B21 <=> (P11 | P22 | P33)
B41 <=> (P33 | P42)
B32 <=> (P33 | P22 | P32)
B22 <=> (P22 | P13 | P24)
B43 <=> (P42 | P33 | P44)
B33 <=> (P33 | P24 | P44)
S12 <=> (W13 | W11 | W22)
S22 <=> (W42 | W22 | W24 | W13)
S14 <=> (W13 | W24)
~P11
~W11
Gold23
(P11 & ~W11)
(P22 & ~P24)


**Memanggil Fungsi untuk Mengecek Keberadaan Emas**

In [17]:
for clause in clauses:
    wumpus_kb.tell(expr(clause))

In [19]:
# Mengecek apakah ada Gold di (2,3)
query = expr('Gold23')
if wumpus_kb.ask_if_true(query):
    print("Ada Gold di (2,3).")
else:
    print("Tidak ada Gold di (2,3).")

Ada Gold di (2,3).


**cek apakah jalur 2,3 aman untuk dilewati**

In [25]:
# Mengecek apakah tidak ada Pit di (2,3)
is_no_pit = wumpus_kb.ask_if_true(query)

# Mengecek apakah tidak ada Wumpus di (2,3)
is_no_wumpus = wumpus_kb.ask_if_true(query)

# Memeriksa apakah lokasi aman
if is_no_pit and is_no_wumpus:
    print("Jalur di (2,3) aman.")
    if wumpus_kb.ask_if_true(query):
        print("Dan ada Gold di (2,3), Anda bisa mengambilnya.")
else:
    print("Jalur di (2,3) tidak aman.")

Jalur di (2,3) aman.
Dan ada Gold di (2,3), Anda bisa mengambilnya.


## Pembuktian Menggunakan Truth Table ##

In [30]:
import itertools
import pandas as pd
# menyimpan semua kalusa yang di simpan dalam KB
clauses = ['P11', 'W11', 'P22', 'P31', 'P13', 'B21', 'B12', 'B23', 'P33', 'P24', 'P42', 'W33', 'W22', 'W42', 'W31', 'S23', 'S21', 'Gold23', 'P23', 'P12', 'W12', 'W23']

In [32]:
# Buat semua kemungkinan kombinasi True/False untuk setiap variabel
combinations = list(itertools.product([True, False], repeat=len(clauses)))

In [34]:
# Fungsi untuk mengevaluasi setiap klausa
def evaluate_clause(assignment):
    P11, W11, P22, P31, P13, B21, B12, B23, P33, P24, P42, W33, W22, W42, W31, S23, S21, Gold23, P23, P12, W12, W23 = assignment  
    return (
        not P11 and 
        not W11 and  
        (not P11 or B21) and 
        (not P22 or B21) and 
        (not P31 or B21) and 
        (P11 or P22 or P31 or not B21) and 
        (not P11 or B12) and  
        (not P13 or B12) and  
        (not P22 or B12) and  
        (P11 or P13 or P22 or not B12) and  
        (not P22 or B23) and  
        (not P13 or B23) and  
        (not P24 or B23) and  
        (not P23 or B23) and  
        (P22 or P13 or P24 or P23 or not B23) and  
        (not P33 or B23) and  
        (not P22 or B23) and  
        (not P42 or B23) and  
        (not P31 or B23) and  
        (P33 or P22 or P42 or P31 or not B23) and  
        (not W33 or S23) and  
        (not W22 or S23) and  
        (not W42 or S23) and  
        (not W31 or S23) and  
        (W33 or W22 or W42 or W31 or not S23) and  
        (not W31 or S21) and  
        (not W11 or S21) and  
        (not W22 or S21) and  
        (W31 or W11 or W22 or not S21) and  
        P13 and  
        W31 and  
        not P22 and 
        not W22 and 
        P33 and  
        not P12 and 
        not W12 and 
        not P23 and 
        not W23 and 
        Gold23  
    )

In [36]:
# Evaluasi setiap kombinasi berdasarkan klausa
results = [evaluate_clause(assignment) for assignment in combinations]

In [38]:
# Buat DataFrame dari kombinasi variabel dan hasil evaluasi
df = pd.DataFrame(combinations, columns=clauses)
df['KB'] = results

In [40]:
# Tampilan DataFrame
df.head(10)

,P11,W11,P22,P31,P13,B21,B12,B23,P33,P24,...,W42,W31,S23,S21,Gold23,P23,P12,W12,W23,KB
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,False
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,True,False
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,False,False,False
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,True,False
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,True,False,False
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,True,False
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,False,False,False,False
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,False
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,False,False


### Cek berapa kombinasi (KB) yang bernilai True

In [42]:
kb = pd.DataFrame(df.KB.value_counts().reset_index())
kb.head()

,KB,count
0,False,4194272
1,True,32


### Dari banyak kombinasi yang di coba, diketahui KB yang bernilai `True` sebanyak `32` 

In [44]:
# Menampilkan Truth Table dari kombinasi yang memiliki KB yang bernilai `True`
kb_true = df[df['KB'] == True]
kb_true.head(10)

,P11,W11,P22,P31,P13,B21,B12,B23,P33,P24,...,W42,W31,S23,S21,Gold23,P23,P12,W12,W23,KB
3670543,False,False,False,True,True,True,True,True,True,True,...,True,True,True,True,True,False,False,False,False,True
3670799,False,False,False,True,True,True,True,True,True,True,...,False,True,True,True,True,False,False,False,False,True
3671567,False,False,False,True,True,True,True,True,True,True,...,True,True,True,True,True,False,False,False,False,True
3671823,False,False,False,True,True,True,True,True,True,True,...,False,True,True,True,True,False,False,False,False,True
3672591,False,False,False,True,True,True,True,True,True,True,...,True,True,True,True,True,False,False,False,False,True
3672847,False,False,False,True,True,True,True,True,True,True,...,False,True,True,True,True,False,False,False,False,True
3673615,False,False,False,True,True,True,True,True,True,True,...,True,True,True,True,True,False,False,False,False,True
3673871,False,False,False,True,True,True,True,True,True,True,...,False,True,True,True,True,False,False,False,False,True
3674639,False,False,False,True,True,True,True,True,True,False,...,True,True,True,True,True,False,False,False,False,True
3674895,False,False,False,True,True,True,True,True,True,False,...,False,True,True,True,True,False,False,False,False,True


### Cek apakah pada kotak `[2,3]` aman dilewati

In [46]:
kb_true_safe = pd.DataFrame(kb_true[['W23', 'P23']].value_counts().reset_index(name='count'))
kb_true_safe

,W23,P23,count
0,False,False,32


### Cek apakah pada kotak [2,3] terdapat emas

In [48]:
kb_true_gold = pd.DataFrame(kb_true['Gold23'].value_counts().reset_index(name='count'))
kb_true_gold

,Gold23,count
0,True,32


### kesimpulan

**Jadi, dari analisis yang sudah kita lakukan, setiap kombinasi tabel kebenaran yang menghasilkan KB True menandakan bahwa koordinat [2,3] aman untuk dilewati dan terdapat emas di lokasi tersebut.**